In [1]:
import os
import pandas as pd
import tqdm

In [2]:
# titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
# titles.columns=['Id', 'Title']

titles = pd.read_csv('./titles_speller_stemmer.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

In [3]:
titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [4]:
#df_speller = pd.DataFrame(columns=['Id', 'Text'])
df_speller_stemmer = pd.DataFrame(columns=['Id', 'Title'])

In [5]:
#lst_speller = []
lst_speller_stemmer = []

In [6]:
for doc in tqdm.tqdm(os.listdir('./DOCS_TEXTS/')):
    if '.ipynb' in doc:
        continue
    doc_id = doc.split('.')[0]
    with open(os.path.join('./DOCS_TEXTS', doc), 'r') as f:
        text_speller = f.readline()
        title_speller_stemmer = f.readline()
        
        #lst_speller.append((doc_id, text_speller))
        lst_speller_stemmer.append([doc_id, title_speller_stemmer])

100%|██████████| 582168/582168 [03:54<00:00, 2485.62it/s]


In [7]:
# df_speller = pd.DataFrame(lst_speller, columns =['Id', 'Text'])
# df_speller['Id'] = df_speller['Id'].apply(lambda x: int(x))
# df_speller = df_speller.sort_values(by=['Id'])

df_speller_stemmer = pd.DataFrame(lst_speller_stemmer, columns =['Id', 'Text'])
df_speller_stemmer['Id'] = df_speller_stemmer['Id'].apply(lambda x: int(x))
df_speller_stemmer = df_speller_stemmer.sort_values(by=['Id'])

In [8]:
# merged = titles.merge(df_speller, left_on=['Id'], right_on=['Id']).sort_values(by=['Id'])
merged = titles.merge(df_speller_stemmer, left_on=['Id'], right_on=['Id']).sort_values(by=['Id'])

In [9]:
# queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
# queries.columns = ['Id', 'Query']
queries = pd.read_csv('./query_speller_stemmer.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']

In [10]:
queries

,Id,Query
0,0,13 причина почему
1,1,1 положительный и 1 отрицательный мочь ли
2,2,2016 действовать ли зао рождественский мануфак...
3,3,1 месяц после операция на кишечник диета что м...
4,4,2 правда 1 ложь что можно придумывать
...,...,...
6306,6306,являться ли тойота харриер внедорожник
6307,6307,як можно очищать креходить
6308,6308,являться ли реактив медицинский изделие
6309,6309,являться ли словообразовательный пары слово го...


In [11]:
queries['Id'] = queries['Id'].apply(lambda x: int(x))

### QA

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [13]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['USE_QA'] = None

In [14]:
titles = merged['Title']
corpus = merged['Text']

In [15]:
all_q = list(sorted(set(sample_df['query_id'])))

In [29]:
for query in tqdm.tqdm(all_q):        
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]
    try:
        question_embeddings = module.signatures['question_encoder'](tf.constant(q))
        responses = titles[docs_id]
        response_contexts = corpus[docs_id]
        response_embeddings = module.signatures['response_encoder'](
            input=tf.constant(responses),
            context=tf.constant(response_contexts))

        sim = np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel()
        sample_df.loc[sample_df['query_id'] == query, 'USE_QA'] = sim
        #sample_df.to_csv('USEQA_Full_Spelled.tsv',sep='\t',index=False)
    except:
        question_embeddings = module.signatures['question_encoder'](tf.constant(q))
        sim = []
        for batch in np.array_split(docs_id, 50):   
            responses = titles[batch]
            response_contexts = corpus[batch]
            response_embeddings = module.signatures['response_encoder'](
                input=tf.constant(responses),
                context=tf.constant(response_contexts))
            sim.extend(np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel())

        sample_df.loc[sample_df['query_id'] == query, 'USE_QA'] = sim
            #sample_df.to_csv('USEQA_Full_Spelled.tsv',sep='\t',index=False)

100%|██████████| 1313/1313 [08:56<00:00,  2.45it/s]


In [30]:
#sample_df.to_csv('USE/USEQA_Full_Spelled.tsv',sep='\t',index=False)
sample_df.to_csv('USE/USEQA_Full_Spelled_Stemmed.tsv',sep='\t',index=False)

## LaReQA

In [16]:
loaded = hub.load("https://tfhub.dev/google/LAReQA/mBERT_X_Y/1")

question_encoder = loaded.signatures["query_encoder"]
response_encoder = loaded.signatures["response_encoder"]

In [17]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['LaReQA'] = None

In [24]:
counter = 0
for query in tqdm.tqdm(all_q):
    counter += 1
    if counter % 100:
        sample_df.to_csv('LaReQA_Full_Spelled.tsv',sep='\t',index=False)

    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]

    question_embeddings = question_encoder(input=tf.constant(np.asarray([q])))["outputs"]

    responses = titles[docs_id]
    response_contexts = corpus[docs_id]
    
    response_embeddings = response_encoder(
        input=tf.constant(responses),
        context=tf.constant(response_contexts))["outputs"]
    
    sim = np.matmul(question_embeddings.numpy(), np.transpose(response_embeddings.numpy()))
    sample_df.loc[sample_df['query_id'] == query, 'LaReQA'] = sim[0]

100%|██████████| 200/200 [06:27<00:00,  1.94s/it]


In [25]:
sample_df.to_csv('LaReQA/LaReQA_Full_Spelled.tsv',sep='\t',index=False)